# Power electricity power consumption prediction model

Let's first start by importing the needed libraries

In [1]:
from pyspark.sql import SparkSession

And create a local parallel spark session

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Power electricity prediction") \
        .getOrCreate()

The we read the dataframe we need to apply the prediction on

In [3]:
train = spark.read.csv('../data/engineered/powerelectricity_train.csv', header=True, inferSchema=True)
test = spark.read.csv('../data/engineered/powerelectricity_test.csv', header=True, inferSchema=True)

Now we will test some regression models and evaluate them using rmse

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [5]:
train_ep = train.drop('rms_current')
train_ep = train_ep.drop('electric_power')

#train_rms = train.drop('electric_power')

test_ep = test.drop('rms_current')
test_ep = test_ep.drop('electric_power')

#test_rms = test.drop('electric_power')

features_ep = train_ep.columns.copy()
#features_ep.remove('electric_power')
features_ep.remove('delta_y')

#features_rms = train_rms.columns.copy()
#features_rms.remove('rms_current')

In [6]:
assembler_pe_train = VectorAssembler(inputCols=features_ep,outputCol="features")
train_ep = assembler_pe_train.transform(train_ep)
assembler_pe_test = VectorAssembler(inputCols=features_ep,outputCol="features")
test_ep = assembler_pe_test.transform(test_ep)

Let's start by predicting electric_power

In [7]:
rf_pe = RandomForestRegressor(featuresCol="features",labelCol='delta_y', maxDepth=15, maxBins=14, minInstancesPerNode=1, minInfoGain=0.0005, subsamplingRate=1, seed=42, numTrees=50)

In [8]:
%%time

model_pe = rf_pe.fit(train_ep)

CPU times: user 23.4 ms, sys: 12.3 ms, total: 35.7 ms
Wall time: 40.3 s


In [9]:
predictions = model_pe.transform(train_ep)

In [10]:
print('################################ TRAIN ################################')
evaluator_rmse = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R squared (r2) on train data = %g" % r2)

evaluator_mae = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Average Error (RMSE) on train data = %g" % mae)

################################ TRAIN ################################
Root Mean Squared Error (RMSE) on train data = 0.0428189
R squared (r2) on train data = 0.976754
Mean Average Error (RMSE) on train data = 0.0315551


In [11]:
predictions = model_pe.transform(test_ep)

In [12]:
print('################################ TEST ################################')

evaluator_rmse = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R squared (r2) on test data = %g" % r2)

evaluator_mae = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Average Error (RMSE) on test data = %g" % mae)

################################ TEST ################################
Root Mean Squared Error (RMSE) on test data = 0.171166
R squared (r2) on test data = 0.496638
Mean Average Error (RMSE) on test data = 0.136756


In [13]:
predictions.select(["delta_y", "prediction"]).show(100000)

+--------------------+--------------------+
|             delta_y|          prediction|
+--------------------+--------------------+
|0.034705441414907945| 0.05782540854078156|
| 0.09087099736165749| 0.05782540854078156|
| 0.09087099736165749| 0.06485314850532116|
| 0.08864514946971269| 0.06464662514108216|
| 0.16771376017374973| 0.06464662514108216|
|  0.1498056870002098| 0.06464662514108216|
| 0.14840408261612437| 0.06464662514108216|
| 0.10847122888506489| 0.05527418163826294|
|-0.01086561383660...| 0.05484350702787319|
|-0.03277450513582...| 0.05059504833246981|
|-0.05828494200223666|0.033441557324509354|
|0.001087899066690...| 0.04952909156142943|
| 0.04709824854708744| 0.06025862363438472|
| 0.09982418931664094| 0.05942412441015457|
|  0.1327745628672874|0.057750172865051706|
| 0.03512307408248061| 0.06686924104247104|
| 0.04240788269484019| 0.05963497771369242|
|0.003853382562441...| 0.05548070500250193|
| 0.01570231318271098|  0.0662320430678423|
|0.006110089330363611| 0.0552741

And finally save to a file the predictions

In [14]:
import os

try:
    os.mkdir('../predictions/')
except:
    pass

preds = predictions.select(["prediction"])
preds.toPandas().to_csv('../predictions/preds.csv')